In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY,
        "X-Goog-Api-Key": GOOGLE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
	)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__
live = client.is_live()
connected = client.is_connected()

print(f"Weaviate Ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"Weaviate Live: {client.is_live()}")
print(f"Client Connected: {connected}")


In [ ]:
# Define the hybrid generative search function
def hybrid_generative_search(client, query, collection_name, grouped_task: str = None, limit: int = 5):
    """Run a hybrid generative search and return a normalized result."""
    
    collection = client.collections.get(collection_name)

    if grouped_task is None:
        grouped_task = (
            "Analyze the context and provide explanations briefly." # This can be customized as needed
        )

    try:
        from weaviate.classes.generate import GenerativeConfig
        
        response = collection.generate.hybrid( 
            query=query,
            grouped_task=grouped_task,
            limit=limit,
            generative_provider=GenerativeConfig.openai(model="gpt-4") # change model as needed
        )

        if not getattr(response, "objects", None):
            class Dummy:
                class Gen:
                    text = (
                        "I couldn't find relevant information in the documents to answer that. "
                        "Try rephrasing or providing more context."
                    )
                generative = Gen()
                objects = []
            return Dummy()

        print(f"Search returned {len(response.objects)} relevant documents")
        return response

    except Exception as e:
        class Err:
            class Gen:
                text = f"Search error: {str(e)}"
            generative = Gen()
            objects = []
        return Err()

result = hybrid_generative_search(client, "<<Query>>", "<Collection_Name>", limit=3)
print("Generative answer:")
print(result.generative.text)

In [ ]:
# Example usage of the generative search function
from weaviate.classes.generate import GenerativeConfig

collection = client.collections.get("<Collection_Name>")  # Replace with your collection name
response = collection.generate.near_text(
    query="<<Query>>",  # Replace with your query
    limit=2,
    grouped_task="Analyse the answer then provide a brief.", # Customize as needed
    generative_provider=GenerativeConfig.openai(model="gpt-4") # change model as needed
)

# Print Generated Answer
if hasattr(response, 'generative') and response.generative:
    print("\nGENERATED ANSWER:")
    print("-" * 60)
    print(response.generative.text)

# Print Retrieved Documents
print("\n" + "=" * 60)
print("RETRIEVED DOCUMENTS")
print("=" * 60)

for i, obj in enumerate(response.objects, 1):
    print(f"\n--- Document {i} ---")
    print(f"File: {obj.properties.get('fileName', 'N/A')}")
    print(f"Chunk: {obj.properties.get('chunkIndex', 'N/A')}")
    print(f"\nContent Preview:")
    content = obj.properties.get('content', '')
    print(content[:300] + "..." if len(content) > 300 else content)

# Close connection
client.close()
print("Connection closed.")